# 残差网络（ResNet）

_随着我们设计越来越深的网络，深刻理解“新添加的层如何提升神经网络的性能”变得至关重要。更重要的是设计网络的能力，在这种网络中，添加层会使网络更具表现力，
为了取得质的突破，我们需要一些数学基础知识。————泛函_  

更大、更深的神经网络可能表征问题的能力更强，但是不总是能够提高模型的精度。  
如果我们把网络能够表现的模型看成一个函数，那么也就是说，更大的网络能够表示的所有函数泛函最大。  
但很有可能，大网络的整个泛函集合都是离最优解远的方向偏移的，有可能造成最优解还不如较简单网络的情况  
> 我们能做的一件事是，让更复杂网络的泛函至少完全包括更简单的泛函，--->以此保证其至少不会比之前更加偏离最优解  

![ResNet 网络设计思路](../img/residual-block.svg)

最终就是希望能够通过各种ResNet的变体来实现对我们的神经网络的重新设计


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        #两个3x3的卷积，1的填充，1的步长，最终输出维度和输入是不变的
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        #作用于通道维的批量归一化层
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)


![ResNet-18 网络架构](../img/resnet18.svg)  
按照这个思路实现网络  
> 当\*参数出现在函数定义时，表示可变参数  
> 当\*参数出现在函数调用时，则表示解包功能


In [ ]:

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))  #因为本质上resnet返回的是一个list，*list就是把这个list本身所有的元素按序拿进来
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))